# Ayudantía 9: Estructuras Nodales II
### Autores:
 - Caua Paz (@csantiagopaz)
 - Valentina Campaña (@aerotecnia99)
 - Pablo Kipreos Palau (@Pablok98)


## Grafos

 - Es un conjunto de nodos y sus relaciones.
 
 - Son una unión entre **listas ligadas y árboles**
 
 - Pueden ser **dirigidos o no dirigidos**
 
   - En un **grafo dirigido**, las relaciones tienen orientación
        - Si A se relaciona con B, no necesariamente B se relaciona con A
        
   - En un **grafo no dirigido**, las relaciones **no** tienen orientación
        - Si A se relaciona con B, **necesariamente** B se relaciona con A



![grafos](img/grafos.png)


### Representación en código

In [1]:
class Nodo:
    # Un nodo está compuesto por el valor que almacena y una lista de sus nodos vecinos
    def __init__(self, valor):
        self.valor = valor
        self.vecinos = []

    def agregar_vecino(self, nodo):
        self.vecinos.append(nodo)

    def __repr__(self):
        texto = f"[{self.valor}]"
        if len(self.vecinos) > 0:
            textos_vecinos = [f"[{vecino.valor}]" for vecino in self.vecinos]
            texto += " -> " + ", ".join(textos_vecinos)
        return texto

In [2]:
# Instanciamos nodos y agregamos conexiones del grafo dirigido

nodo_1 = Nodo(1)
nodo_2 = Nodo(2)
nodo_3 = Nodo(3)
nodo_4 = Nodo(4)
nodo_5 = Nodo(5)
nodo_1.agregar_vecino(nodo_2)
nodo_2.agregar_vecino(nodo_3)
nodo_3.agregar_vecino(nodo_2)
nodo_3.agregar_vecino(nodo_4)
nodo_3.agregar_vecino(nodo_5)
nodo_4.agregar_vecino(nodo_5)

In [3]:
print(nodo_1)
print(nodo_2)
print(nodo_3)
print(nodo_4)
print(nodo_5)

[1] -> [2]
[2] -> [3]
[3] -> [2], [4], [5]
[4] -> [5]
[5]


## Lista de adyacencia
Es simplemente una representación compacta de las relaciones entre vértices. Sirve para tener acceso a la información del grafo y simplificar las operaciones.

In [8]:
# Aquí usamos diccionarios "int-list" porque ofrece más facilidad de búsqueda.
# Cada llave del diccionario es el valor de un vértice, y el valor asociado es la lista de vertices con conexión.

grafo_no_dirigido = {1: [2],
                     2: [1, 3],
                     3: [2, 4, 5],
                     4: [3, 5],
                     5: [3, 4]
                    }

grafo_dirigido = {1: [2],
                  2: [3],
                  3: [2, 4, 5],
                  4: [5],
                  5: []
                 }

In [9]:
# ¿Vértices del grafo?
print(list(grafo_no_dirigido.keys()))

# ¿Conexiones de vértice 3 en grafo no dirigido?
print(grafo_no_dirigido[3])

[1, 2, 3, 4, 5]
[2, 4, 5]


## Operaciones de grafos

Digamos que tenemos la siguiente clase `Grafo`, representando los nodos con una lista de adyacencia.



In [10]:
class Grafo:

    def __init__(self, lista_adyacencia=None):
        self.lista_adyacencia = lista_adyacencia or dict()

Podemos implementar las siguientes operaciones básicas:

- `adyacentes(G, x, y)`: Indica si existe una arista entre x e y. Retorna un booleano.

In [8]:
    def adyacentes(self, x, y):
        return (y in self.lista_adyacencia[x])

- `vecinos(G, x)`: Entrega una lista con todos los vértices `y` tales que existe una arista entre `x` e `y`. Retorna una lista de nodos.

In [9]:
    def vecinos(self, x):
            return self.lista_adyacencia[x]

- `agregar_vertice(G, x)`: Agrega el vértice `x` al grafo.

In [10]:
    def agregar_vertice(self, x):
            self.lista_adyacencia[x] = set()

- `remover_vertice(G, x)`: Elimina el vértice `x` del grafo.

In [11]:
    def remover_vertice(self, x):
        # Remueve el nodo x de la lista de adyacencia
        self.lista_adyacencia.pop(x, None)
        # Remueve el nodo x de las conexiones de los nodos restantes del grafo
        for k, v in self.lista_adyacencia.items():
            if x in v:
                v.remove(x)

- `agregar_arista(G, x, y)`: Agrega una arista entre los vértices x e y.

In [12]:
    def agregar_arista(self, x, y):
        if x in self.lista_adyacencia:
            self.lista_adyacencia[x].add(y)

- `remover_arista(G, x, y)`: Elimia la arista entre x e y.

In [13]:
    def remover_arista(self, x, y):
        vecinos_x = self.lista_adyacencia.get(x, set())
        # Remueve el nodo y de los vecinos de x
        if y in vecinos_x:
            vecinos_x.remove(y)

## Recorrido de grafos

### BFS (Breadth-first search)
 - Es un recorrido de **amplitud**. A grandes rasgos, se tienen que visitar todos los vecinos de un nivel para pasar al siguiente.
 - Utiliza una **cola** para mantener registro de los nodos **por visitar**.
 - El algoritmo parte con una cola compuesta por el nodo inicial.
 - Cada vez que se visita un nodo, se agregan a la cola todos los vecinos de este.
 - Es útil cuando se quiere encontrar **la cantidad mínima de aristas a recorrer** o el camino más corto entre nodos.

![bfs](https://i.imgur.com/Csvfiho.gif)

Veamos el ejemplo de los contenidos del curso:

In [14]:
from collections import deque


def bfs(grafo, inicio):
    # Vamos a mantener una lista con los nodos visitados.
    visitados = []
    # La cola de siempre, comienza desde el nodo inicio.
    queue = deque([inicio])

    while len(queue) > 0:
        # Elegimos el siguiente nodo a visitar de la cola
        vertice = queue.popleft()
        # Detalle clave: si ya visitamos el nodo, no hacemos nada!
        if vertice in visitados:
            continue

        # Lo visitamos
        print(vertice)
        visitados.append(vertice)
        # Agregamos los vecinos a la cola si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                queue.append(vecino)
    return visitados

### DFS (Depth-first search)
 - Es un recorrido en **profundidad**. A grandes rasgos, se va visitando el primer vecino de cada nivel hasta llegar al último nivel posible, es decir, intentando llegar lo más lejos que puede antes de devolverse.
 - Utiliza un **stack** para mantener registro de los nodos **por visitar**.
 - Su versión iterativa es muy parecida a BFS, el orden de recorrido cambia al utilizar un **stack** en vez de un **queue**.
 - Es útil para encontrar rápidamente una solución cuando sabemos de antemano que es "profunda", por ejemplo, cuando se tiene que encontrar un camino entre dos puntos y dicho camino recorre muchos nodos.

![dfs](https://i.imgur.com/VQWZmya.gif)

Veamos el ejemplo de los contenidos del curso:

#### Versión iterativa

In [5]:
def dfs(grafo, inicio):
    # Vamos a mantener un set con los nodos visitados.
    visitados = set()
    # El stack de siempre, comienza desde el nodo inicio.
    stack = [inicio]

    while len(stack) > 0:
        vertice = stack.pop()
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vertice in visitados:
            continue
        # Lo visitamos
        print(vertice)
        visitados.add(vertice)

        # Agregamos los vecinos al stack si es que no han sido visitados.
        for vecino in grafo[vertice]:
            if vecino not in visitados:
                stack.append(vecino)

    return list(visitados)

#### Versión recursiva

In [4]:
# Vamos a mantener como parámetro un set con los nodos visitados.
def dfs_recursivo(grafo, vertice, visitados=None):
    visitados = visitados or set()

    # Lo visitamos
    print(vertice)
    visitados.add(vertice)

    for vecino in grafo[vertice]:
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vecino not in visitados:
            dfs_recursivo(grafo, vecino, visitados)

    return list(visitados)

**Cabe destacar que ambos métodos recorren exhaustivamente el grafo, dado un punto de partida.** Por lo que si un nodo no fue visitado en el recorrido, significa que **no es alcanzable** desde ese punto de partida.

## Ejercicio propuesto
Ahora vamos a aplicar los conceptos de grafo y BFS en un ejemplo concreto

### Ahora haremos una versión más corta de la actividad sumativa 03 del 2020-2

En el contexto de la pandemia el programa consiste representar las redes de aeropuertos del mundo como un grafo dirigido. Los aeropuertos tienen conexiones entre sí, y cada una de ellas tiene un cierto riesgo asociado, representado
como un número de **_infectados_**. Este número equivale a una estimación (realizada por el departamento
de Probabilidades y Estadísticas) sobre la cantidad de infectados que puede generar un vuelo desde un
aeropuerto hacia otro.

Al iniciar el programa, se poblará el grafo, representado en RedesMundiales, con los datos iniciales. Luego se deberá agregar los Aeropuertos candidatos
y sus Conexiones.

A continuación se deja una figura que representa el grafo con todas las redes que
se encuentran en RedesMundiales y los infectados en cada conexión, a partir de los datos reales de los
archivos entregados.

![](img/graph.png "Graph")

A partir de la figura anterior podemos ver que los nodos corresponden a los aeropuertos y las aristas a las conexiones entre los aeropuertos. Por lo que es posible definir dos tipos de entidades: la clase Aeropuerto y la clase Conexion.

In [ ]:
class Aeropuerto:
    def __init__(self, aeropuerto_id, nombre):
        self.id = aeropuerto_id
        self.nombre = nombre
        self.conexiones = []

    def __repr__(self):
        return f"ID del aeropuerto:{self.id}"


class Conexion:
    def __init__(self, aeropuerto_inicio_id, aeropuerto_llegada_id, infectados):

        self.aeropuerto_inicio_id = aeropuerto_inicio_id
        self.aeropuerto_llegada_id = aeropuerto_llegada_id
        self.infectados = infectados

    def __repr__(self):
        return f"{self.aeropuerto_inicio_id}->{self.aeropuerto_llegada_id}"

Por otra parte, el grafo es modelado en la clase RedesMundiales y que contiene las acciones que se pueden realizar como por ejemplo agregar/eliminar nodos y conexiones.

In [17]:
class RedesMundiales:

    def __init__(self):
        # Estructura donde se guardaran los aeropuertos
        # Cada llave es un id y el valor es una instancia de Aeropuerto
        self.aeropuertos = {}

In [18]:
 def agregar_aeropuerto(self, aeropuerto_id, nombre):

        agregar_aeropuerto = Aeropuerto(aeropuerto_id, nombre)
        self.aeropuertos[agregar_aeropuerto.id] = agregar_aeropuerto

In [19]:
 def cargar_red(self, ruta_aeropuertos, ruta_conexiones):

        # Primero se crean todos los aeropuertos
        for aeropuerto_id, nombre in cargar_aeropuertos(ruta_aeropuertos):
            self.agregar_aeropuerto(aeropuerto_id, nombre)

        # Después generamos las conexiones
        for id_partida, id_salida, infectados in cargar_conexiones(ruta_conexiones):
            self.agregar_conexion(id_partida, id_salida, infectados)

```python
def agregar_conexion(self, aeropuerto_id_partida, aeropuerto_id_llegada, infectados):
```
Este método recibe aeropuerto_id_partida, que es el id del Aeropuerto de partida de la conexión
y aeropuerto_id_llegada, que es el `id` del de llegada. Deberás completar este método, tal que se
verifique que ambos aeropuertos se encuentran en self.aeropuertos y que la conexión no existe.
En caso de cumplirse estas condiciones, se deberá crear la conexión entre ambos Aeropuertos. Para
esto basta actualizar las listas conexiones del Aeropuerto de partida.


In [20]:
def agregar_conexion(self, aeropuerto_id_partida, aeropuerto_id_llegada, infectados):
    partida = aeropuerto_id_partida
    llegada = aeropuerto_id_llegada

    aeropuerto_partida = self.aeropuertos.get(partida)
    aeropuerto_llegada = self.aeropuertos.get(llegada)

    if aeropuerto_partida and aeropuerto_llegada:
        conexion = Conexion(partida, llegada, infectados)

        if conexion not in aeropuerto_partida.conexiones:
            aeropuerto_partida.conexiones.append(conexion)



In [21]:
def eliminar_conexion(self, conexion):
        id_partida = conexion.aeropuerto_inicio_id
        id_llegada = conexion.aeropuerto_llegada_id
        aeropuerto_inicio = self.aeropuertos.get(id_partida)
        for conexion in aeropuerto_inicio.conexiones:
            if conexion.aeropuerto_llegada_id == id_llegada:
                aeropuerto_inicio.conexiones.remove(c)
                break


```python
def escala_mas_corta(self, id_aeropuerto_1, id_aeropuerto_2):
```
 Este método recibe los identificadores para un aeropuerto de partida y un aeropuerto de llegada. Este método puede retornar
o simplemente imprimir las partes de la ruta más corta posible entre estos dos aeropuertos (puedes
retornar/imprimir los nodos o las conexiones). En caso, de que no se pueda llegar del aeropuerto
de partida al aeropuerto de llegada, no deberás, retornar ni imprimir nada. **Puedes agregar la
cantidad de argumentos que te parezcan necesarios para completar este método.**

In [22]:
#Version Recursiva, se puede hacer iterativa tambien, es a gusto del programador
def escala_mas_corta(self, aeropuerto1_id, aeropuerto2_id, lista_visitados = [], ruta = [], distancia = float('inf')):
    aeropuerto_1 = self.aeropuertos[aeropuerto1_id]
    aeropuerto_2 = self.aeropuertos[aeropuerto2_id]
    self.ruta = ruta
    self.distancia = distancia
    
    for conexion in aeropuerto_1.conexiones:
        aeropuerto = self.aeropuertos.get(conexion.aeropuerto_llegada_id)
        if aeropuerto not in lista_visitados:
            lista_visitados.append(aeropuerto)
            if aeropuerto == aeropuerto_2:
                if len(lista_visitados) < self.distancia and lista_visitados:
                    self.distancia = len(lista_visitados)
                    self.ruta = lista_visitados
                    print(self.ruta)
            elif aeropuerto.conexiones:
                self.escala_mas_corta(aeropuerto.id, aeropuerto_2.id, lista_visitados, self.ruta, self.distancia)
            lista_visitados.remove(aeropuerto)
    return self.ruta

## Ejercicio Propuesto II

Resolveremos un sudoku de 4 filas y 4 columnas, es decir, menos celdas utilizando el método DFS.

In [6]:
class Nodo:
    def __init__(self, columna, fila, valor):
        self.fila = fila
        self.columna = columna
        self.valor = valor
        self.vecinos = []
    
    def __repr__(self):
        return self.columna + str(self.fila)

In [7]:
class Grafo:
    def __init__(self):
        self.celdas = {}
    
    def __repr__(self):
        completo = ""
        for i in range(1,5):
            a = self.celdas["A"+str(i)].valor
            b = self.celdas["B"+str(i)].valor
            c = self.celdas["C"+str(i)].valor
            d = self.celdas["D"+str(i)].valor
            fila = f"| {a} | {b} | {c} | {d} |\n"
            completo += fila
        return completo

In [8]:
sudoku = Grafo()

# Agregamos los nodos que en este caso son las celdas
sudoku.celdas["A1"] = Nodo("A", 1, 1)
sudoku.celdas["A2"] = Nodo("A", 2, 3)
sudoku.celdas["A3"] = Nodo("A", 3, 4)
sudoku.celdas["A4"] = Nodo("A", 4, "-")
sudoku.celdas["B1"] = Nodo("B", 1, 2)
sudoku.celdas["B2"] = Nodo("B", 2, "-")
sudoku.celdas["B3"] = Nodo("B", 3, 3)
sudoku.celdas["B4"] = Nodo("B", 4, 1)
sudoku.celdas["C1"] = Nodo("C", 1, 4)
sudoku.celdas["C2"] = Nodo("C", 2, "-")
sudoku.celdas["C3"] = Nodo("C", 3, "-")
sudoku.celdas["C4"] = Nodo("C", 4, 3)
sudoku.celdas["D1"] = Nodo("D", 1, "-")
sudoku.celdas["D2"] = Nodo("D", 2, 2)
sudoku.celdas["D3"] = Nodo("D", 3, 1)
sudoku.celdas["D4"] = Nodo("D", 4, 4)

# Agregamos las conexiones entre nodos, 
# en este caso es no dirigido ya que una celda está conectada con todos los nodos de su misma fila, columna y cuadrante.
for celda in sudoku.celdas.values():
    for vecino in sudoku.celdas.values():
        if celda.fila == vecino.fila and celda.columna == vecino.columna:
            pass
        elif celda.fila == vecino.fila:
            celda.vecinos.append(vecino)
        elif celda.columna == vecino.columna:
            celda.vecinos.append(vecino)
for celda in sudoku.celdas.values():
    print(f"{celda.columna}{celda.fila} tiene vecinos a {celda.vecinos}")

print(sudoku)

A1 tiene vecinos a [A2, A3, A4, B1, C1, D1]
A2 tiene vecinos a [A1, A3, A4, B2, C2, D2]
A3 tiene vecinos a [A1, A2, A4, B3, C3, D3]
A4 tiene vecinos a [A1, A2, A3, B4, C4, D4]
B1 tiene vecinos a [A1, B2, B3, B4, C1, D1]
B2 tiene vecinos a [A2, B1, B3, B4, C2, D2]
B3 tiene vecinos a [A3, B1, B2, B4, C3, D3]
B4 tiene vecinos a [A4, B1, B2, B3, C4, D4]
C1 tiene vecinos a [A1, B1, C2, C3, C4, D1]
C2 tiene vecinos a [A2, B2, C1, C3, C4, D2]
C3 tiene vecinos a [A3, B3, C1, C2, C4, D3]
C4 tiene vecinos a [A4, B4, C1, C2, C3, D4]
D1 tiene vecinos a [A1, B1, C1, D2, D3, D4]
D2 tiene vecinos a [A2, B2, C2, D1, D3, D4]
D3 tiene vecinos a [A3, B3, C3, D1, D2, D4]
D4 tiene vecinos a [A4, B4, C4, D1, D2, D3]
| 1 | 2 | 4 | - |
| 3 | - | - | 2 |
| 4 | 3 | - | 1 |
| - | 1 | 3 | 4 |



Ahora para terminar de llenar el sudoku debemos implementar la búsqueda DFS y así lograr completarlo!

In [17]:
def dfs_recursivo(grafo, vertice, visitados=None):
    visitados = visitados or set()

    # Lo visitamos
    print(vertice)
    visitados.add(vertice)
    
    for vecino in grafo[vertice.columna + str(vertice.fila)].vecinos:
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vecino not in visitados:
            dfs_recursivo(grafo, vecino, visitados)
            if vertice.valor == "-":
                values = [1, 2, 3, 4]
                for vecino in vertice.vecinos:
                    if vecino.valor in values:
                        values.remove(vecino.valor)
                vertice.valor = values[0]


    return list(visitados)

print(dfs_recursivo(sudoku.celdas, sudoku.celdas["D2"]))
print(sudoku)

D2
A2
A1
A3
A4
B4
B1
B2
B3
C3
C1
C2
C4
D4
D1
D3
[D4, C3, D3, A2, B1, C2, C1, A1, B4, D2, A3, B2, D1, B3, C4, A4]
| 1 | 2 | 4 | 3 |
| 3 | 4 | 1 | 2 |
| 4 | 3 | 2 | 1 |
| 2 | 1 | 3 | 4 |



In [18]:
def dfs_iterativo(grafo, vertice):
    visitados = set()
    por_visitar = [vertice]
    print(vertice)
    visitados.add(vertice)
    while por_visitar:
        visitado = por_visitar.pop()
    for vecino in grafo[visitado.columna + str(visitado.fila)].vecinos:
        # Detalle clave: si ya visitamos el nodo, ¡no hacemos nada!
        if vecino not in visitados:
            if visitado.valor == "-":
                values = [1, 2, 3, 4]
                for vecino in visitado.vecinos:
                    if vecino.valor in values:
                        values.remove(vecino.valor)
                visitado.valor = values[0]

    return list(visitados)

print(dfs_recursivo(sudoku.celdas, sudoku.celdas["D2"]))
print(sudoku)

D2
A2
A1
A3
A4
B4
B1
B2
B3
C3
C1
C2
C4
D4
D1
D3
[D4, C3, D3, A2, B1, C2, C1, A1, B4, D2, A3, B2, D1, B3, C4, A4]
| 1 | 2 | 4 | 3 |
| 3 | 4 | 1 | 2 |
| 4 | 3 | 2 | 1 |
| 2 | 1 | 3 | 4 |

